this notebook calculates the similarity scores between words of the craving corpus via word vectors from Gensim

In [1]:
import numpy as np
import pandas as pd
import gensim.downloader as api
text8_path = api.load('text8', return_path=True)
print("Using corpus from", text8_path)

Using corpus from C:\Users\TheaK/gensim-data\text8\text8.gz


In [2]:
tweets = pd.read_csv("tweets_080123.csv") #use all_words.csv alternatively

tweets

c:\Users\TheaK\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,possibly_sensitive,Tweets,source,lang,created_at,id,author,Branche,Branche-6,Sektor,...,quotes,quote_ids,retweet_ids,media_keys,poll_ids,cashtags,entity_types,entity_names,mentioned_usernames,mentioned_hashtags
0,False,The Berlin State of Mind.\n\nEnjoy the true de...,erased6981162,en,2018-06-09T16:00:13.000Z,1005479690621026306,MercedesBenz,automobil,Automobil,2.0,...,3,[],[],['3_1005479688762994691'],NaN,NaN,"['Organization', 'Other', 'Person', 'Organizat...","['Berlin State', 'Mind', 'Constantin Schiller'...",[],"['MBsocialcar', 'MercedesBenz']"
1,False,It's officially #SportscarTogether day! How ar...,Twitter Web Client,en,2018-06-09T15:56:29.000Z,1005478751809961986,Porsche,automobil,Automobil,2.0,...,3,[],[],['7_1005478556346941443'],NaN,NaN,"['Other', 'Other']","['SportscarTogether', 'Porsche70Years']",[],"['SportscarTogether', 'Porsche70Years']"
2,False,With #Germany experiencing usually high temper...,Percolate,en,2018-06-09T13:22:03.000Z,1005439886764044290,MunichRe,versicherung,Finance/Insurance,3.0,...,0,[],[],{},NaN,NaN,"['Place', 'Other', 'Person', 'Other']","['Germany', 'ExtremeWeather', 'Eberhard Faust'...",[],"['Germany', 'ExtremeWeather', 'climatechange',..."
3,False,The @FIAFormulaE comes to Zurich! Learn five t...,erased6981162,en,2018-06-09T13:04:15.000Z,1005435405473406976,MercedesBenz,automobil,Automobil,2.0,...,3,[],[],['13_1005434382600065026'],NaN,NaN,"['Place', 'Other', 'Other', 'Other', 'Other', ...","['Zurich', 'Mercedes EQ', 'Allianz E-\n Villag...",['FIAFormulaE'],"['SwitchToEQ', 'EQ', 'ABBFormulaE', 'ZurichEpr..."
4,False,"RT @MercedesAMGF1: Friday, that’s a wrap... 🎬 ...",Twitter Web Client,en,2018-06-09T08:28:41.000Z,1005366057174032385,MercedesBenz,automobil,Automobil,2.0,...,0,[],['1005201393806970881'],{},NaN,NaN,"['Person', 'Person', 'Other']","['Valtteri', 'Lewis', 'CanadianGP']",['MercedesAMGF1'],['CanadianGP']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306126,False,It's still not too late to meet the Brenntag B...,Twitter Web Client,en,2018-05-23T07:30:16.000Z,999190762117107712,Brenntag,chemie,Technik,2.0,...,0,[],[],['3_999190729376153600'],{},NaN,"['Other', 'Other', 'Place', 'Place', 'Other', ...","['Brenntag Brazil Cosmetics Team', 'FCE Cosmet...",[],"['fcecosmetique', 'BrenntagBrazil']"
306127,False,RT @adidasrunning: There is a new dawn for run...,Twitter Ads,en,2018-05-22T22:20:56.000Z,999052517123108864,adidas,mode,Textil,2.0,...,0,[],['999038802822807552'],{},{},NaN,['Other'],['SOLARBOOST'],['adidasrunning'],['SOLARBOOST']
306128,False,Are your tires properly inflated? Now is a goo...,Emplifi,en,2018-05-22T19:30:38.000Z,999009662685986816,continentaltire,automobil,Automobil,2.0,...,0,[],[],['3_999009661046083584'],{},NaN,['Other'],['NationalTireSafetyWeek'],[],['NationalTireSafetyWeek']
306129,False,In order to sustain our planet's finite resour...,Twitter Ads,en,2018-05-22T18:47:00.000Z,998998679565451265,covestro,chemie,Technik,2.0,...,2,[],[],['13_998939017172455429'],{},NaN,['Other'],['UTECHEurope'],[],"['InnovationForSustainableGrowth', 'UTECHEurope']"


Preprocessing

In [3]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

tokenizer = RegexpTokenizer(r'\w+')
tweets['Tweets'].dropna(inplace=True)
tweets['Tweets'] = tweets['Tweets'].str.lower()
tweets['post_tokens'] = tweets['Tweets'].astype(str).apply(tokenizer.tokenize)

# Make a list of english stopwords
stopwords = nltk.corpus.stopwords.words("english")
tweets['post_tokens'] = tweets['post_tokens'].apply(lambda x: [item for item in x if item not in stopwords])

wordnet_lem = WordNetLemmatizer()

tweets['post_lemmas'] = tweets['post_tokens'].apply(lambda l: [wordnet_lem.lemmatize(x) for x in l])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TheaK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\TheaK\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\TheaK\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\TheaK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
words = tweets['post_lemmas'].values.tolist() 
tweets

,possibly_sensitive,Tweets,source,lang,created_at,id,author,Branche,Branche-6,Sektor,...,retweet_ids,media_keys,poll_ids,cashtags,entity_types,entity_names,mentioned_usernames,mentioned_hashtags,post_tokens,post_lemmas
0,False,the berlin state of mind.\n\nenjoy the true de...,erased6981162,en,2018-06-09T16:00:13.000Z,1005479690621026306,MercedesBenz,automobil,Automobil,2.0,...,[],['3_1005479688762994691'],NaN,NaN,"['Organization', 'Other', 'Person', 'Organizat...","['Berlin State', 'Mind', 'Constantin Schiller'...",[],"['MBsocialcar', 'MercedesBenz']","[berlin, state, mind, enjoy, true, definition,...","[berlin, state, mind, enjoy, true, definition,..."
1,False,it's officially #sportscartogether day! how ar...,Twitter Web Client,en,2018-06-09T15:56:29.000Z,1005478751809961986,Porsche,automobil,Automobil,2.0,...,[],['7_1005478556346941443'],NaN,NaN,"['Other', 'Other']","['SportscarTogether', 'Porsche70Years']",[],"['SportscarTogether', 'Porsche70Years']","[officially, sportscartogether, day, celebrati...","[officially, sportscartogether, day, celebrati..."
2,False,with #germany experiencing usually high temper...,Percolate,en,2018-06-09T13:22:03.000Z,1005439886764044290,MunichRe,versicherung,Finance/Insurance,3.0,...,[],{},NaN,NaN,"['Place', 'Other', 'Person', 'Other']","['Germany', 'ExtremeWeather', 'Eberhard Faust'...",[],"['Germany', 'ExtremeWeather', 'climatechange',...","[germany, experiencing, usually, high, tempera...","[germany, experiencing, usually, high, tempera..."
3,False,the @fiaformulae comes to zurich! learn five t...,erased6981162,en,2018-06-09T13:04:15.000Z,1005435405473406976,MercedesBenz,automobil,Automobil,2.0,...,[],['13_1005434382600065026'],NaN,NaN,"['Place', 'Other', 'Other', 'Other', 'Other', ...","['Zurich', 'Mercedes EQ', 'Allianz E-\n Villag...",['FIAFormulaE'],"['SwitchToEQ', 'EQ', 'ABBFormulaE', 'ZurichEpr...","[fiaformulae, comes, zurich, learn, five, thin...","[fiaformulae, come, zurich, learn, five, thing..."
4,False,"rt @mercedesamgf1: friday, that’s a wrap... 🎬 ...",Twitter Web Client,en,2018-06-09T08:28:41.000Z,1005366057174032385,MercedesBenz,automobil,Automobil,2.0,...,['1005201393806970881'],{},NaN,NaN,"['Person', 'Person', 'Other']","['Valtteri', 'Lewis', 'CanadianGP']",['MercedesAMGF1'],['CanadianGP'],"[rt, mercedesamgf1, friday, wrap, valtteri, le...","[rt, mercedesamgf1, friday, wrap, valtteri, le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306126,False,it's still not too late to meet the brenntag b...,Twitter Web Client,en,2018-05-23T07:30:16.000Z,999190762117107712,Brenntag,chemie,Technik,2.0,...,[],['3_999190729376153600'],{},NaN,"['Other', 'Other', 'Place', 'Place', 'Other', ...","['Brenntag Brazil Cosmetics Team', 'FCE Cosmet...",[],"['fcecosmetique', 'BrenntagBrazil']","[still, late, meet, brenntag, brazil, cosmetic...","[still, late, meet, brenntag, brazil, cosmetic..."
306127,False,rt @adidasrunning: there is a new dawn for run...,Twitter Ads,en,2018-05-22T22:20:56.000Z,999052517123108864,adidas,mode,Textil,2.0,...,['999038802822807552'],{},{},NaN,['Other'],['SOLARBOOST'],['adidasrunning'],['SOLARBOOST'],"[rt, adidasrunning, new, dawn, running, see, s...","[rt, adidasrunning, new, dawn, running, see, s..."
306128,False,are your tires properly inflated? now is a goo...,Emplifi,en,2018-05-22T19:30:38.000Z,999009662685986816,continentaltire,automobil,Automobil,2.0,...,[],['3_999009661046083584'],{},NaN,['Other'],['NationalTireSafetyWeek'],[],['NationalTireSafetyWeek'],"[tires, properly, inflated, good, time, check,...","[tire, properly, inflated, good, time, check, ..."
306129,False,in order to sustain our planet's finite resour...,Twitter Ads,en,2018-05-22T18:47:00.000Z,998998679565451265,covestro,chemie,Technik,2.0,...,[],['13_998939017172455429'],{},NaN,['Other'],['UTECHEurope'],[],"['InnovationForSustainableGrowth', 'UTECHEurope']","[order, sustain, planet, finite, resources, de...","[order, sustain, planet, finite, resource, de

Train Gensim word2vec model

In [5]:
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.word2vec import Text8Corpus

# Using params from Word2Vec_FastText_Comparison

params = {
    'alpha': 0.05,
    'vector_size': 200, #size
    'window': 10,
    'epochs': 5, #iter
    'min_count': 10,#for craving 30 or 50
    'sample': 1e-4,
    'sg': 1,
    'hs': 0,
    'negative': 10,
}

model = Word2Vec(sentences = words, **params)
#model = Word2Vec(Text8Corpus(text8_path), **params)
wv = model.wv
print("Using trained model", wv)

Using trained model KeyedVectors<vector_size=200, 25223 keys>


In [8]:
check = ['future', 'green', 'work','value']
main_word = 'sustainability'
for word in check:
  print(main_word, '<>', word, wv.similarity(main_word, word))

sustainability <> future 0.33479264
sustainability <> green 0.3891449
sustainability <> work 0.3226266
sustainability <> value 0.3938291


In [12]:
#pip install annoy
from gensim.similarities.annoy import AnnoyIndexer

# 100 trees are being used in this example
annoy_index = AnnoyIndexer(model, 100)


In [17]:
similar_terms = []
print_annoy_sims = False
print_normal_sims = False
concepts = ["sustainable", "sustainability", "green", "future", "acting", "climate"]
for concept in concepts:
    df = pd.DataFrame()
    vector = wv[concept]
    if(print_annoy_sims):
        # The instance of AnnoyIndexer we just created is passed, AnnoyIndexer is an approximate neighbour algorithm by Spotify that is faster, we prefer the other for this use case though
        approximate_neighbors = wv.most_similar([vector], topn=20, indexer=annoy_index)
        # Neatly print the approximate_neighbors and their corresponding cosine similarity values
        print("Approximate Neighbors (via AnnoyIndexer by Spotify)")
        for neighbor in approximate_neighbors:
            print("%-*s  Occ.: %s" % (35,neighbor,wv.get_vecattr(neighbor[0], "count")))

    normal_neighbors = wv.most_similar([vector], topn=30)
    if(print_normal_sims):
        print("\nExact Neighbors (via Gensim)")
    for neighbor in normal_neighbors:
        df = df.append({concept : neighbor[0], concept+'_sim' : round(neighbor[1],3), concept+'_count' : wv.get_vecattr(neighbor[0], "count")}, ignore_index = True)
        if(print_normal_sims):
            print("%-*s  Occ.: %s" % (35,neighbor,wv.get_vecattr(neighbor[0], "count")))
    similar_terms.append(df)  

similarities = pd.concat(similar_terms, axis=1)
similarities

,sustainable,sustainable_sim,sustainable_count,sustainability,sustainability_sim,sustainability_count,green,green_sim,green_count,future,future_sim,future_count,acting,acting_sim,acting_count,climate,climate_sim,climate_count
0,sustainable,1.000,3684.0,sustainability,1.000,4208.0,green,1.000,1931.0,future,1.000,9261.0,acting,1.000,53.0,climate,1.000,2276.0
1,sustainability,0.694,4208.0,sustainable,0.694,3684.0,envy,0.530,20.0,democratization,0.560,10.0,urgency,0.519,13.0,climatechange,0.745,625.0
2,sust,0.673,12.0,sust,0.658,12.0,viper,0.528,15.0,technology,0.537,8585.0,actonclimate,0.502,10.0,cfigueres,0.652,10.0
3,sdg11,0.652,15.0,agsr,0.646,11.0,stephensimpson1,0.495,15.0,smar,0.533,10.0,gcleaderssummit,0.471,44.0,ipcc,0.646,23.0
4,fraukekuehn,0.643,10.0,ethibel,0.642,17.0,lead,0.492,1913.0,quentinclark,0.528,10.0,podobnik,0.460,13.0,mcii,0.641,15.0
5,guardianhenkel,0.630,12.0,sb16copenhagen,0.636,21.0,contifestival,0.486,10.0,cso32,0.526,20.0,politicoeurope,0.452,10.0,ipcc_ch,0.637,10.0
6,worldcitiesday,0.625,12.0,ethical_corp,0.627,13.0,northeastgp,0.486,20.0,sustainable,0.522,3684.0,persil,0.449,57.0,globalwarming,0.631,33.0
7,engages,0.621,11.0,unga,0.627,12.0,flag,0.485,212.0,anjahoffmann,0.520,12.0,erinmeez,0.445,11.0,wmo,0.631,10.0
8,agsr,0.619,11.0,voeste,0.623,20.0,jackhawkrace,0.485,10.0,personalised,0.520,20.0,anja_rechenberg,0.440,17.0,climateweek,0.619,10.0
9,sb16copenhagen,0.617,21.0,constituent,0.623,10.0,goin,0.473,13.0,afnsyhj7qt,0.519,12.0,anchored,0.440,11.0,cwnyc,0.617,36.0


In [18]:
similarities.to_csv('similar_terms.csv', header=True, index=False, columns=list(similarities.axes[1]))

In [20]:
print("vocabulary length is:")
print(len(wv))

vocabulary length is:
25223
